In [1]:
from tqdm import tqdm
import torch
import os
import sys
sys.path.append("/scratch/m000122/bcabrown/continual-learning")
sys.path.append("/scratch/m000122/bcabrown/continual-learning/third_party/cartridges")
os.environ["CARTRIDGES_DIR"] = "/scratch/m000122/bcabrown/continual-learning/third_party/cartridges"
os.environ["CARTRIDGES_OUTPUT_DIR"] = "/scratch/m000122/bcabrown/continual-learning/third_party/cartridges/output"
from third_party.cartridges.cartridges.datasets import read_conversations
from third_party.cartridges.cartridges.data.finer.evals import LLAMA_CARTRIDGE_TEMPLATE
import torch.nn.functional as F
import math

/scratch/m000122/bcabrown/miniconda/envs/brad/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/scratch/m000122/bcabrown/miniconda/envs/brad/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/scratch/m000122/bcabrown/miniconda/envs/brad/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: Unsupport

In [2]:
# data_path = "/scratch/m000122/stalaei/logs/continual_learning/data/finer_v1_train_ICL_exclude_current_250_triplets_false_1000_reps_temp_0.7/dataset.jsonl"
data_path = "/scratch/m000122/bcabrown/cl_data/finer_v1_train_ICL_exclude_current_250_triplets_false_1000_reps_temp_0.7_subset.jsonl"
data = read_conversations(data_path)

In [3]:
row = data[0]

In [4]:
num_output_ids = len(row["output_ids"])
for idx in range(num_output_ids):
    topk_token_ids = row["topk_token_ids"][idx]
    topk_token_probs = row["topk_logprobs"][idx]
    output_id = row["output_ids"][idx]
    try:
        rank = topk_token_ids.index(output_id)
        print(output_id, rank, math.exp(topk_token_probs[rank]))
    except ValueError:
        idx = -1
        print(output_id, "not in topk", len(topk_token_ids))
    

1271 0 1.0
8417 0 1.0
279 0 1.0
8475 0 1.0
75760 0 0.9999998807907033
4833 0 1.0
2326 0 1.0
27438 0 1.0
2599 0 0.9999998807907033
4877 0 1.0
369 0 1.0
10768 0 0.9999997615814209
5120 0 1.0
13 0 1.0
21 0 1.0
19727 0 0.9999988079071329
304 0 1.0
279 0 1.0
2728 0 1.0
11914 0 1.0
11 0 1.0
584 0 1.0
1205 0 1.0
311 0 1.0
10765 0 1.0
279 0 1.0
7434 0 1.0
1694 0 1.0
7633 0 1.0
13 0 1.0
763 0 1.0
420 0 1.0
1162 0 1.0
11 0 1.0
279 0 1.0
7434 0 1.0
374 0 0.9999960660934519
279 0 1.0
18361 0 0.677715365867456
7318 0 0.9994500279510469
369 0 0.9981015325116461
264 0 0.9886732700445328
2626 0 0.9999991655349802
10824 0 0.9997735023492937
382 0 0.9456622011018311
59 0 0.9999998807907033
80175 0 1.0
90 0 1.0
23562 0 0.8948144349051357
37292 0 0.9998880624779843
2617 0 1.0
11916 1 0.12299285966216365
17647 0 1.0
11948 0 0.9999926090240764
15289 0 1.0
82 0 1.0
92 0 0.9999960660934519
128009 0 0.9999719858164584


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.75it/s]


In [7]:
model.cuda().to(torch.bfloat16)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
  

In [8]:
prompt_tokens = tokenizer.apply_chat_template(
    row["teacher_messages"], 
    add_generation_prompt=True, 
    chat_template=LLAMA_CARTRIDGE_TEMPLATE, 
    add_special_tokens=False, 
    tokenize=True
)
# prompt_tokens = tokenizer.apply_chat_template(
#     row["input_messages"], 
#     add_generation_prompt=True, 
#     chat_template=LLAMA_CARTRIDGE_TEMPLATE, 
#     add_special_tokens=False, 
#     tokenize=True
# )
prompt_and_answer_tokens = prompt_tokens + row["output_ids"]
num_answer_tokens = len(row["output_ids"])

In [9]:
len(prompt_and_answer_tokens)

46850

In [10]:
print(tokenizer.decode(prompt_and_answer_tokens))

<|start_header_id|>system<|end_header_id|>

You will be solving a series of financial tagging tasks. At every step, you will be given a sentence and asked to identify the appropriate XBRL US GAAP tag for a term in the sentence. After this, you will receive feedback on if your tag was correct or incorrect. For every task, use your previous attempts and corresponding feedback when solving problems to continuously improve.

Here is a list of US GAAP tags options: OperatingLeasesRentExpenseNet,MinorityInterestOwnershipPercentageByParent,ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsGrantsInPeriodWeightedAverageGrantDateFairValue,DerivativeNotionalAmount,PreferredStockDividendRatePercentage,GuaranteeObligationsMaximumExposure,LossContingencyEstimateOfPossibleLoss,OperatingLeaseRightOfUseAsset,NumberOfOperatingSegments,PaymentsToAcquireBusinessesNetOfCashAcquired,DebtInstrumentBasisSpreadOnVariableRate1,InterestExpense,ShareBasedCompensationArrange

In [11]:
with torch.no_grad():
    input_ids = torch.tensor(prompt_and_answer_tokens).cuda().unsqueeze(0)
    outputs = model(input_ids)

In [12]:
answer_logits = outputs.logits[:, -num_answer_tokens-1:-1]

In [13]:
answer_logits.shape

torch.Size([1, 58, 128256])

In [14]:
for answer_idx, (hf_token_logits, toka_topk_logprobs, toka_topk_ids) in enumerate(zip(answer_logits[0], row["topk_logprobs"], row["topk_token_ids"], strict=True)):
    hf_logprobs = F.softmax(hf_token_logits, dim=-1).log()
    hf_topk_logprobs, hf_topk_ids = torch.topk(hf_logprobs, k=10, dim=-1)
    hf_topk_ids = hf_topk_ids.tolist()
    hf_topk_logprobs = hf_topk_logprobs.tolist()
    print("HuggingFace Top 5 IDs and LogProbs:")
    for id, logprob in zip(hf_topk_ids[:5], hf_topk_logprobs[:5]):
        token = tokenizer.decode([id])
        print(f"  Token: {token!r:>12}, ID: {id:6d}, Prob: {math.exp(logprob):.4f}")
    
    print("\nToka Top 5 IDs and LogProbs:") 
    for id, logprob in zip(toka_topk_ids[:5], toka_topk_logprobs[:5]):
        token = tokenizer.decode([id])
        print(f"  Token: {token!r:>12}, ID: {id:6d}, Prob: {math.exp(logprob):.4f}")
    
    print("="*30)

HuggingFace Top 5 IDs and LogProbs:
  Token:         'To', ID:   1271, Prob: 1.0000
  Token:        'The', ID:    791, Prob: 0.0000
  Token:         'to', ID:    998, Prob: 0.0000
  Token:    'Correct', ID:  34192, Prob: 0.0000
  Token:        '_to', ID:   2401, Prob: 0.0000

Toka Top 5 IDs and LogProbs:
  Token:         'To', ID:   1271, Prob: 1.0000
  Token:        'The', ID:    791, Prob: 0.0000
  Token:         'to', ID:    998, Prob: 0.0000
  Token:        ' To', ID:   2057, Prob: 0.0000
  Token:        '_to', ID:   2401, Prob: 0.0000
HuggingFace Top 5 IDs and LogProbs:
  Token: ' determine', ID:   8417, Prob: 1.0000
  Token: ' determined', ID:  11075, Prob: 0.0000
  Token: ' determining', ID:  26679, Prob: 0.0000
  Token: ' determines', ID:  27667, Prob: 0.0000
  Token: ' Determine', ID:  31001, Prob: 0.0000

Toka Top 5 IDs and LogProbs:
  Token: ' determine', ID:   8417, Prob: 1.0000
  Token: ' determined', ID:  11075, Prob: 0.0000
  Token: ' determining', ID:  26679, Prob: 0.00